#  Inférence de l'effet - Stratégie Multilabels - Approche deep Learning
Dans ce Notebook, nous cosntruisons un modèle qui permet d'inférer l'EFFET à partir de la classification de l'incident et des données textuelles en ce basant sur des reseau récurents commes GRU/LSTM etc.

En effet, ces approches ont montré des réultats très encourageant et nous voulons explorer cette direction pour peut être faire des réceau recurent notre modèle par défault.

Nous considérons ce problème comme un problème de classification multiclasses et multilabels. En effet, il y a plusieurs effets possibles et un incidents peut entrainer plusieurs effets.

Dans ce note book nous nous posons les questions suivantes : 
- Quel est l'impact du drop out ?
- Rajouter des couches augmentent-ils les performaces ?
- L'utilisation de réseaux bidirectionnel est-elle pertinente ?
- Une couche d'attention est-elle utile ?
- Attention is all we need, really ?
- Utilisation des embeddings 
- Concaténation des modèles sur différentes entrées ?

In [7]:
import pandas as pd
import numpy as np
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,SpatialDropout1D, Bidirectional,SimpleRNN,Input, concatenate, Reshape
import tensorflow 

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from keras.layers import Concatenate, GlobalMaxPool1D, Dropout
from tensorflow.keras.layers import Attention

from sklearn.metrics import  precision_score, recall_score, f1_score

tensorflow.random.set_seed(1234)

## 1. Chargement des données

In [3]:
mlb = MultiLabelBinarizer()

train = pd.read_pickle('./data_split/train.pkl')
# Pour faire un modèle sans le 
#train = train[~train['TEF_ID'].map(lambda x : 106 in x)]
X_train = train[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','ETAT_PATIENT','ACTION_PATIENT']]
y_train = mlb.fit_transform(train['CDY_ID'])
test =  pd.read_pickle('./data_split/test.pkl')
#test = test[~test['TEF_ID'].map(lambda x : k in x)]
X_test = test[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','ETAT_PATIENT','ACTION_PATIENT']]
y_test = mlb.transform(test['CDY_ID'])


X_train_dgs = np.load('results/dgs_camenbert_train_vec.npy')
X_test_dgs =np.load('results/dgs_camenbert_test_vec.npy')



df_effets = pd.read_csv("data/ref_MRV/referentiel_dispositif_effets_connus.csv",delimiter=';',encoding='ISO-8859-1')
df_dys = pd.read_csv("data/ref_MRV/referentiel_dispositif_dysfonctionnement.csv",delimiter=';',encoding='ISO-8859-1')

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:995: UserWarning: unknown class(es) [55] will be ignored
  .format(sorted(unknown, key=str)))


## 2.1 LSTM et TFIDF, une première baseline

In [4]:
%%time
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
import spacy
nlp =spacy.load('fr')
from spacy.lang.fr.stop_words import STOP_WORDS

preprocess = ColumnTransformer(
    [('description_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 10000,norm = 'l2'), 'DESCRIPTION_INCIDENT'),
     
     ('etat_pat_tfidf', TfidfVectorizer(sublinear_tf=True, min_df=3,ngram_range=(1, 1),
                                       stop_words=STOP_WORDS,
                                       max_features = 10000,norm = 'l2'), 'ETAT_PATIENT'),
     ('action_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 5000,norm = 'l2'), 'ACTION_PATIENT'),
     
     
     ('fabricant_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 5000,norm = 'l2'), 'FABRICANT')
     ],
    
    remainder='passthrough')

X_train_, X_test_ =preprocess.fit_transform(X_train),preprocess.transform(X_test)

from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000)
X_train_ = svd.fit_transform(X_train_)
X_test_ = svd.transform(X_test_)


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['qu', 'quelqu'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['qu', 'quelqu'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['qu', 'quelqu'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocess

CPU times: user 3min 34s, sys: 3min 1s, total: 6min 35s
Wall time: 1min 33s


In [5]:
X_train_ = np.reshape(X_train_, (X_train_.shape[0], 1, X_train_.shape[1]))
X_test_ = np.reshape(X_test_, (X_test_.shape[0], 1, X_test_.shape[1]))

In [8]:
model = Sequential()
model.add(LSTM(200))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

epochs = 10
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 13s 628us/step - loss: 0.0324 - categorical_accuracy: 0.8051 - val_loss: 0.0313 - val_categorical_accuracy: 0.7630
Epoch 2/10
21059/21059 [==============================] - 12s 580us/step - loss: 0.0203 - categorical_accuracy: 0.8245 - val_loss: 0.0298 - val_categorical_accuracy: 0.7578
Epoch 3/10
21059/21059 [==============================] - 13s 606us/step - loss: 0.0182 - categorical_accuracy: 0.8280 - val_loss: 0.0299 - val_categorical_accuracy: 0.7485
Epoch 4/10
21059/21059 [==============================] - 12s 558us/step - loss: 0.0169 - categorical_accuracy: 0.8330 - val_loss: 0.0297 - val_categorical_accuracy: 0.7445
Epoch 5/10
21059/21059 [==============================] - 10s 479us/step - loss: 0.0159 - categorical_accuracy: 0.8382 - val_loss: 0.0303 - val_categorical_accuracy: 0.7379
Epoch 6/10
21059/21059 [==============================] - 9s 447us/step - loss: 0.0151

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.5070, Recall: 0.9602, F1-measure: 0.6260
For threshold:  0.04
Samples-average quality numbers
Precision: 0.6795, Recall: 0.9289, F1-measure: 0.7527
For threshold:  0.06
Samples-average quality numbers
Precision: 0.7203, Recall: 0.9172, F1-measure: 0.7790
For threshold:  0.08
Samples-average quality numbers
Precision: 0.7457, Recall: 0.9094, F1-measure: 0.7947
For threshold:  0.1
Samples-average quality numbers
Precision: 0.7628, Recall: 0.9009, F1-measure: 0.8042
For threshold:  0.12
Samples-average quality numbers
Precision: 0.7748, Recall: 0.8920, F1-measure: 0.8098
For threshold:  0.14
Samples-average quality numbers
Precision: 0.7847, Recall: 0.8852, F1-measure: 0.8143
For threshold:  0.16
Samples-average quality numbers
Precision: 0.7939, Recall: 0.8794, F1-measure: 0.8185
For threshold:  0.2


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.8036, Recall: 0.8652, F1-measure: 0.8200
For threshold:  0.25
Samples-average quality numbers
Precision: 0.8131, Recall: 0.8509, F1-measure: 0.8211
For threshold:  0.3
Samples-average quality numbers
Precision: 0.8187, Recall: 0.8377, F1-measure: 0.8192
For threshold:  0.35
Samples-average quality numbers
Precision: 0.8214, Recall: 0.8232, F1-measure: 0.8152
For threshold:  0.4
Samples-average quality numbers
Precision: 0.8199, Recall: 0.8110, F1-measure: 0.8096
For threshold:  0.5
Samples-average quality numbers
Precision: 0.8030, Recall: 0.7806, F1-measure: 0.7879
For threshold:  0.6
Samples-average quality numbers
Precision: 0.7611, Recall: 0.7449, F1-measure: 0.7502
For threshold:  0.7
Samples-average quality numbers
Precision: 0.7167, Recall: 0.7051, F1-measure: 0.7089


## Commentaires : 

Nous obtenons un score de base line de 0.6730. Nous allons essayer d'améliorer ce score avec difféntes approches : 
- Remplacer le tfidf par un embedding entrainé par le modèle 
    - Embedding simple
    - Embedding par colonnes
    
- Essayer un modèle biLSTM ou GRU ou bi GRU
- Essayer de rajouter une couche d'attention dans Keras


In [9]:
model = Sequential()
model.add(LSTM(200, dropout=0.15))
model.add(Dense(y_train.shape[1], activation='softmax'))



model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['categorical_accuracy'])

epochs = 4
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/4
21059/21059 [==============================] - 14s 688us/step - loss: 0.0276 - categorical_accuracy: 0.8105 - val_loss: 0.0298 - val_categorical_accuracy: 0.7592
Epoch 2/4
21059/21059 [==============================] - 14s 688us/step - loss: 0.0194 - categorical_accuracy: 0.8204 - val_loss: 0.0299 - val_categorical_accuracy: 0.7605
Epoch 3/4
21059/21059 [==============================] - 14s 660us/step - loss: 0.0177 - categorical_accuracy: 0.8254 - val_loss: 0.0299 - val_categorical_accuracy: 0.7489
Epoch 4/4
6580/6580 [==============================] - 0s 74us/step
loss :  0.019804676357758625
categorical accuracy:  0.81686931848526
####################################
For threshold:  0.01
Samples-average quality numbers
Precision: 0.3990, Recall: 0.9731, F1-measure: 0.5411
For threshold:  0.04
Samples-average quality numbers
Precision: 0.6366, Recall: 0.9380, F1-measure: 0.7260
For threshold:  0.06
Samples-average quality nu

# 2.2 Comment performe un reseau de BiLSTM  ?


In [10]:
from keras.layers import Bidirectional

In [11]:
model = Sequential()
model.add(Bidirectional(LSTM(200, dropout=0.1)))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

epochs = 5
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/5
21059/21059 [==============================] - 13s 598us/step - loss: 0.0303 - categorical_accuracy: 0.8072 - val_loss: 0.0305 - val_categorical_accuracy: 0.7654
Epoch 2/5
21059/21059 [==============================] - 17s 799us/step - loss: 0.0198 - categorical_accuracy: 0.8212 - val_loss: 0.0296 - val_categorical_accuracy: 0.7573
Epoch 3/5
21059/21059 [==============================] - 20s 933us/step - loss: 0.0179 - categorical_accuracy: 0.8257 - val_loss: 0.0296 - val_categorical_accuracy: 0.7404
Epoch 4/5
21059/21059 [==============================] - 19s 891us/step - loss: 0.0167 - categorical_accuracy: 0.8300 - val_loss: 0.0299 - val_categorical_accuracy: 0.7455
Epoch 5/5
6580/6580 [==============================] - 2s 270us/step
loss :  0.01992118165497088
categorical accuracy:  0.8158054947853088
####################################
For threshold:  0.01
Samples-average quality numbers
Precision: 0.4467, Recall: 0.9695,

## Commentaire ?

L'augementation de la performance est très minime : 0.6733 ?
Nous remarquons que notre modèle sur apprend assez rapidement, nous allons essayer d'ajouter une couche de dropout pour limiter ce sur apprentissage.
## Si on ajoute du dropout ?

In [12]:
model = Sequential()
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.1)))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation="elu"))
model.add(Dropout(0.1))
model.add( Dense(y_train.shape[1], activation="softmax"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


epochs = 10
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 22s 1ms/step - loss: 0.0270 - categorical_accuracy: 0.8091 - val_loss: 0.0324 - val_categorical_accuracy: 0.7632
Epoch 2/10
21059/21059 [==============================] - 17s 823us/step - loss: 0.0200 - categorical_accuracy: 0.8202 - val_loss: 0.0301 - val_categorical_accuracy: 0.7478
Epoch 3/10
21059/21059 [==============================] - 15s 715us/step - loss: 0.0183 - categorical_accuracy: 0.8253 - val_loss: 0.0298 - val_categorical_accuracy: 0.7474
Epoch 4/10
21059/21059 [==============================] - 16s 750us/step - loss: 0.0171 - categorical_accuracy: 0.8281 - val_loss: 0.0304 - val_categorical_accuracy: 0.7538
Epoch 5/10
21059/21059 [==============================] - 20s 966us/step - loss: 0.0162 - categorical_accuracy: 0.8364 - val_loss: 0.0303 - val_categorical_accuracy: 0.7314
Epoch 6/10
21059/21059 [==============================] - 20s 952us/step - loss: 0.0155 

## Commentaire ?

Les performances changent très peu,  nous allons essayer d'augementer la profondeur de notre reseau.

## Si on ajoute plusieurs couches ?

In [13]:
from tensorflow.keras import regularizers
reg = regularizers.l2(1e-4)
model = Sequential()
model.add(Bidirectional(LSTM(256,  return_sequences=True, dropout=0.1)))
model.add((SimpleRNN(128,return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation="softmax"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


epochs = 10
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 28s 1ms/step - loss: 0.0261 - categorical_accuracy: 0.8098 - val_loss: 0.0302 - val_categorical_accuracy: 0.7597
Epoch 2/10
21059/21059 [==============================] - 25s 1ms/step - loss: 0.0195 - categorical_accuracy: 0.8202 - val_loss: 0.0293 - val_categorical_accuracy: 0.7487
Epoch 3/10
21059/21059 [==============================] - 26s 1ms/step - loss: 0.0178 - categorical_accuracy: 0.8254 - val_loss: 0.0298 - val_categorical_accuracy: 0.7379
Epoch 4/10
21059/21059 [==============================] - 23s 1ms/step - loss: 0.0168 - categorical_accuracy: 0.8317 - val_loss: 0.0310 - val_categorical_accuracy: 0.7521
Epoch 5/10
21059/21059 [==============================] - 22s 1ms/step - loss: 0.0159 - categorical_accuracy: 0.8370 - val_loss: 0.0308 - val_categorical_accuracy: 0.7238
Epoch 6/10
21059/21059 [==============================] - 19s 883us/step - loss: 0.0152 - catego

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.5059, Recall: 0.9548, F1-measure: 0.6248
For threshold:  0.04
Samples-average quality numbers
Precision: 0.6771, Recall: 0.9259, F1-measure: 0.7505
For threshold:  0.06
Samples-average quality numbers
Precision: 0.7175, Recall: 0.9139, F1-measure: 0.7764
For threshold:  0.08
Samples-average quality numbers
Precision: 0.7404, Recall: 0.9048, F1-measure: 0.7900
For threshold:  0.1
Samples-average quality numbers
Precision: 0.7579, Recall: 0.8972, F1-measure: 0.8000
For threshold:  0.12
Samples-average quality numbers
Precision: 0.7701, Recall: 0.8879, F1-measure: 0.8055
For threshold:  0.14
Samples-average quality numbers
Precision: 0.7792, Recall: 0.8795, F1-measure: 0.8089
For threshold:  0.16
Samples-average quality numbers
Precision: 0.7881, Recall: 0.8737, F1-measure: 0.8126
For threshold:  0.2


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.7993, Recall: 0.8619, F1-measure: 0.8160
For threshold:  0.25
Samples-average quality numbers
Precision: 0.8078, Recall: 0.8470, F1-measure: 0.8160
For threshold:  0.3
Samples-average quality numbers
Precision: 0.8135, Recall: 0.8341, F1-measure: 0.8146
For threshold:  0.35
Samples-average quality numbers
Precision: 0.8169, Recall: 0.8224, F1-measure: 0.8120
For threshold:  0.4
Samples-average quality numbers
Precision: 0.8176, Recall: 0.8095, F1-measure: 0.8073
For threshold:  0.5
Samples-average quality numbers
Precision: 0.8017, Recall: 0.7773, F1-measure: 0.7853
For threshold:  0.6
Samples-average quality numbers
Precision: 0.7591, Recall: 0.7420, F1-measure: 0.7476
For threshold:  0.7
Samples-average quality numbers
Precision: 0.7122, Recall: 0.7002, F1-measure: 0.7041


### Commentaires : 

Encore une fois, le modèle semble avoir atteint un plafond de verre qui ne soit pas dépassable ni par la prodondeur, ni par l'ajout de droptout. Nous allons essayer d'ajouter une couche d'attention.

## Une couche d'attention ?

In [15]:
from keras_self_attention import SeqSelfAttention
import keras

model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(keras.layers.Flatten(name='Flatten'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


epochs = 10
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.26,0.27,0.28,0.29,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 13s 613us/step - loss: 0.1145 - categorical_accuracy: 0.7885 - val_loss: 0.0346 - val_categorical_accuracy: 0.7605
Epoch 2/10
21059/21059 [==============================] - 14s 678us/step - loss: 0.0251 - categorical_accuracy: 0.8139 - val_loss: 0.0319 - val_categorical_accuracy: 0.7643
Epoch 3/10
21059/21059 [==============================] - 12s 588us/step - loss: 0.0218 - categorical_accuracy: 0.8249 - val_loss: 0.0303 - val_categorical_accuracy: 0.7633
Epoch 4/10
21059/21059 [==============================] - 12s 577us/step - loss: 0.0196 - categorical_accuracy: 0.8267 - val_loss: 0.0296 - val_categorical_accuracy: 0.7635
Epoch 5/10
21059/21059 [==============================] - 15s 695us/step - loss: 0.0182 - categorical_accuracy: 0.8303 - val_loss: 0.0294 - val_categorical_accuracy: 0.7578
Epoch 6/10
21059/21059 [==============================] - 12s 547us/step - loss: 0.017

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.4609, Recall: 0.9706, F1-measure: 0.5900
For threshold:  0.04
Samples-average quality numbers
Precision: 0.6513, Recall: 0.9399, F1-measure: 0.7353
For threshold:  0.06
Samples-average quality numbers
Precision: 0.6984, Recall: 0.9288, F1-measure: 0.7671
For threshold:  0.08
Samples-average quality numbers
Precision: 0.7259, Recall: 0.9197, F1-measure: 0.7843
For threshold:  0.1


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.7444, Recall: 0.9119, F1-measure: 0.7951
For threshold:  0.12
Samples-average quality numbers
Precision: 0.7586, Recall: 0.9044, F1-measure: 0.8026
For threshold:  0.14
Samples-average quality numbers
Precision: 0.7692, Recall: 0.8983, F1-measure: 0.8081
For threshold:  0.16
Samples-average quality numbers
Precision: 0.7787, Recall: 0.8935, F1-measure: 0.8132
For threshold:  0.2
Samples-average quality numbers
Precision: 0.7914, Recall: 0.8821, F1-measure: 0.8184
For threshold:  0.25
Samples-average quality numbers
Precision: 0.8022, Recall: 0.8703, F1-measure: 0.8216
For threshold:  0.26
Samples-average quality numbers
Precision: 0.8043, Recall: 0.8683, F1-measure: 0.8222
For threshold:  0.27
Samples-average quality numbers
Precision: 0.8057, Recall: 0.8666, F1-measure: 0.8226
For threshold:  0.28
Samples-average quality numbers
Precision: 0.8070, Recall: 0.8641, F1-measure: 0.8227
For threshold:  0.29
Samples-average quality numbers
Precis

## Commentaire : 
Les résultats sembles similaire. Toutefois, la loss de validation est bien meilleur sans pour autant augmenter le f1 score. Nous allons essayer de faire seulement un réseau basé sur l'attention. Nous utilison ici, une verion multiplicative de l'attention.

In [16]:
import keras
from keras_self_attention import SeqSelfAttention

inputs = keras.layers.Input(shape=(1,1000))
#lstm = keras.layers.Bidirectional(keras.layers.LSTM(units=256,
                                                    #return_sequences=True))(inputs)
att = SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       kernel_regularizer=keras.regularizers.l2(1e-4),
                       bias_regularizer=keras.regularizers.l1(1e-4),
                       attention_regularizer_weight=1e-4,
                       name='Attention')(inputs)

flatten = keras.layers.Flatten(name='Flatten')(att)
dense = keras.layers.Dense(units=y_train.shape[1], name='Dense')(flatten)

model = keras.models.Model(inputs=inputs, outputs=[dense])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


epochs = 20
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/20
21059/21059 [==============================] - 12s 593us/step - loss: 0.0712 - categorical_accuracy: 0.5526 - val_loss: 0.0561 - val_categorical_accuracy: 0.6699
Epoch 2/20
21059/21059 [==============================] - 12s 575us/step - loss: 0.0403 - categorical_accuracy: 0.7873 - val_loss: 0.0496 - val_categorical_accuracy: 0.7398
Epoch 3/20
21059/21059 [==============================] - 12s 574us/step - loss: 0.0340 - categorical_accuracy: 0.8035 - val_loss: 0.0462 - val_categorical_accuracy: 0.7432
Epoch 4/20
21059/21059 [==============================] - 13s 631us/step - loss: 0.0304 - categorical_accuracy: 0.8037 - val_loss: 0.0443 - val_categorical_accuracy: 0.7516
Epoch 5/20
21059/21059 [==============================] - 12s 552us/step - loss: 0.0280 - categorical_accuracy: 0.8043 - val_loss: 0.0436 - val_categorical_accuracy: 0.7548
Epoch 6/20
21059/21059 [==============================] - 12s 568us/step - loss: 0.026

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.5845, Recall: 0.9364, F1-measure: 0.6883
For threshold:  0.04
Samples-average quality numbers
Precision: 0.6370, Recall: 0.9291, F1-measure: 0.7251
For threshold:  0.06
Samples-average quality numbers
Precision: 0.6680, Recall: 0.9242, F1-measure: 0.7460
For threshold:  0.08
Samples-average quality numbers
Precision: 0.6937, Recall: 0.9199, F1-measure: 0.7629
For threshold:  0.1


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.7152, Recall: 0.9148, F1-measure: 0.7765
For threshold:  0.12
Samples-average quality numbers
Precision: 0.7332, Recall: 0.9098, F1-measure: 0.7874
For threshold:  0.14
Samples-average quality numbers
Precision: 0.7473, Recall: 0.9034, F1-measure: 0.7952
For threshold:  0.16
Samples-average quality numbers
Precision: 0.7595, Recall: 0.8979, F1-measure: 0.8018
For threshold:  0.2
Samples-average quality numbers
Precision: 0.7778, Recall: 0.8873, F1-measure: 0.8109
For threshold:  0.25
Samples-average quality numbers
Precision: 0.7932, Recall: 0.8714, F1-measure: 0.8161
For threshold:  0.3
Samples-average quality numbers
Precision: 0.8026, Recall: 0.8562, F1-measure: 0.8170
For threshold:  0.35
Samples-average quality numbers
Precision: 0.8122, Recall: 0.8459, F1-measure: 0.8197
For threshold:  0.4
Samples-average quality numbers
Precision: 0.8152, Recall: 0.8332, F1-measure: 0.8171
For threshold:  0.5
Samples-average quality numbers
Precision

### Commentaire : 

Le modèle semble réellement bloqué a une valeure seuil de 0.67. Que ce soit des réseau recurent ou bien des mécanismes d'attention le score reste identique après 10 epochs. Nous allons donc changer de représentation car nous pensons avoir atteint les limite de la représentation en tfidf+SVD.

## Et si l'on changait de représentation ? Utilisation d'Embedding globaux

In [17]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [18]:
df_train = train['DESCRIPTION_INCIDENT']+'. '+train['ETAT_PATIENT']#+ '. '+train['ACTION_PATIENT']
df_test = test['DESCRIPTION_INCIDENT']+'. '+test['ETAT_PATIENT']#+'. '+test['ACTION_PATIENT']

In [19]:
# The maximum number of words to be used. 
MAX_NB_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 300
# This is fixed.
EMBEDDING_DIM = 300

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(df_train.values)
word_index = tokenizer.word_index
print(len(word_index))

X_train = tokenizer.texts_to_sequences(df_train.values)
X_test = tokenizer.texts_to_sequences(df_test.values)
word2index_inputs =  tokenizer.word_index

X_train = pad_sequences(X_train,MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test,MAX_SEQUENCE_LENGTH)
print(X_train.shape,X_test.shape)

61213
(26324, 300) (6580, 300)


In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(200))
model.add(Dense(y_train.shape[1],activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

epochs = 10
batch_size = 64

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)



y_pred = model.predict(X_test)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

/home/user1/DGS-env/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 754s 36ms/step - loss: 0.0293 - categorical_accuracy: 0.8019 - val_loss: 0.0347 - val_categorical_accuracy: 0.7586
Epoch 2/10
21059/21059 [==============================] - 824s 39ms/step - loss: 0.0244 - categorical_accuracy: 0.8111 - val_loss: 0.0344 - val_categorical_accuracy: 0.7605
Epoch 3/10
21059/21059 [==============================] - 829s 39ms/step - loss: 0.0208 - categorical_accuracy: 0.8249 - val_loss: 0.0343 - val_categorical_accuracy: 0.7567
Epoch 4/10
21059/21059 [==============================] - 815s 39ms/step - loss: 0.0169 - categorical_accuracy: 0.8441 - val_loss: 0.0341 - val_categorical_accuracy: 0.7404
Epoch 5/10
21059/21059 [==============================] - 800s 38ms/step - loss: 0.0133 - categorical_accuracy: 0.8703 - val_loss: 0.0384 - val_categorical_accuracy: 0.7185
Epoch 6/10
14208/21059 [===================>..........] - ETA: 4:06 - loss: 0.0101 - c

### Commentaire : 
Les embedding appris n'offre pas de meilleurs résultats pour l'instant. Nous allons essayer de créer des embeddings par colonnes de texte, cette solution c'etait avérée fructueuse dans le cas du tfidf.

## Et si l'on séparait les entrées ?

In [ ]:
# Model constants.
EMBEDDING_DIM =300
MAX_SEQUENCE_LENGTH =300
MAX_NB_WORDS = 50000

def vectorize(df_train,df_test,MAX_NB_WORDS,MAX_SEQUENCE_LENGTH ):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(df_train.values)
    word_index = tokenizer.word_index
    print(len(word_index))

    X_train = tokenizer.texts_to_sequences(df_train.values)
    X_test = tokenizer.texts_to_sequences(df_test.values)
    word2index_inputs =  tokenizer.word_index

    X_train = pad_sequences(X_train,MAX_SEQUENCE_LENGTH)
    X_test = pad_sequences(X_test,MAX_SEQUENCE_LENGTH)
    return (X_train, X_test)
TRAIN = []
for col in ['DESCRIPTION_INCIDENT', 'ETAT_PATIENT', 'FABRICANT'] : 
    X_train,X_test = vectorize(train[col],test[col],MAX_NB_WORDS,MAX_SEQUENCE_LENGTH )
    TRAIN.append((X_train,X_test))

In [ ]:
inputs_1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
inputs_2 = Input(shape=(MAX_SEQUENCE_LENGTH,))
inputs_3 = Input(shape=(MAX_SEQUENCE_LENGTH,))
#x = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs)
#x = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_1)
x = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_1)
x = SpatialDropout1D(0.2)(x)
x = LSTM(200)(x)


#y = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_2)
y = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_2)
y = SpatialDropout1D(0.2)(y)
y = LSTM(200)(y)


#z = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_3)
z = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_3)
z = SpatialDropout1D(0.2)(z)
z = LSTM(200)(z)


w = concatenate([x, y, z])

out =  Dense(y_train.shape[1],activation='softmax')(w)

model = keras.models.Model(inputs=[inputs_1,inputs_2,inputs_3], outputs=out)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

model.summary()


In [ ]:
history = model.fit([TRAIN[0][0],TRAIN[1][0],TRAIN[2][0]], y_train, epochs=10, validation_split=0.2, verbose=1, batch_size = 64)

In [ ]:
y_pred = model.predict([TRAIN[0][1],TRAIN[1][1],TRAIN[2][1]])

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

# Commentaire :
Nous observons que notre modèle sur apprend très rapidement et de manière importante. nous avons deux solutions classiques pour contrer cet effet : 
- Regularisation
- Drop Out
- Netoyer les données avec clean text

In [ ]:
import spacy
nlp =spacy.load('fr')
from spacy.lang.fr.stop_words import STOP_WORDS

import clean_text
stop_words = STOP_WORDS


def vectorize(df_train,df_test,MAX_NB_WORDS,MAX_SEQUENCE_LENGTH,stopword = False ):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    if stopword==True : 
        df_train = df_train.map(lambda x: clean_text.preprocess_text(x))
        df_train = df_train.map(lambda x: [item for item in x.split(" ") if item not in stop_words])
    
    tokenizer.fit_on_texts(df_train.values)
    word_index = tokenizer.word_index
    print(len(word_index))

    X_train = tokenizer.texts_to_sequences(df_train.values)
    X_test = tokenizer.texts_to_sequences(df_test.values)
    word2index_inputs =  tokenizer.word_index

    X_train = pad_sequences(X_train,MAX_SEQUENCE_LENGTH)
    X_test = pad_sequences(X_test,MAX_SEQUENCE_LENGTH)
    return (X_train, X_test)
TRAIN = []
for col in ['DESCRIPTION_INCIDENT', 'ETAT_PATIENT', 'FABRICANT'] : 
    X_train,X_test = vectorize(train[col],test[col],MAX_NB_WORDS,MAX_SEQUENCE_LENGTH,stopword=True )
    TRAIN.append((X_train,X_test))

In [ ]:
inputs_1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
inputs_2 = Input(shape=(MAX_SEQUENCE_LENGTH,))
inputs_3 = Input(shape=(MAX_SEQUENCE_LENGTH,))
#x = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs)
#x = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_1)
x = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_1)
x = SpatialDropout1D(0.2)(x)
x = LSTM(200)(x)


#y = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_2)
y = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_2)
y = SpatialDropout1D(0.2)(y)
y = LSTM(200)(y)


#z = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_3)
z = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_3)
z = SpatialDropout1D(0.2)(z)
z = LSTM(200)(z)


w = concatenate([x, y, z])

out =  Dense(y_train.shape[1],activation='softmax')(w)

model = keras.models.Model(inputs=[inputs_1,inputs_2,inputs_3], outputs=out)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model.summary())

In [ ]:
history = model.fit([TRAIN[0][0],TRAIN[1][0],TRAIN[2][0]], y_train, epochs=5, validation_split=0.2, verbose=1, batch_size = 64)

y_pred = model.predict([TRAIN[0][1],TRAIN[1][1],TRAIN[2][1]])

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

# Conclusion : 
 Suite aux diverses expériences que nousa vosn mener dans ce notebook, nos conclusions sont : 
- La tf-idf est une représentation riche qui est difficile à battre tant en termes de performances qu'en temps de calcul.
- Les modèles d'attention régularisés évitent mieux le sur-apprentissage que les modèles recurent avec du drop out
- séparer les collones pour encoder offre de meillers résultats mais toujours moins bon que ce proposé par le TF-iDF
- Lef ait d'enlever les stop_words n'est pas 


Ce que nous devons essayer : 
- Multi head attention https://www.kaggle.com/fareise/multi-head-self-attention-for-text-classification, https://github.com/CyberZHG/keras-multi-head
- Hierarchical attention : https://paperswithcode.com/paper/hierarchical-attentional-hybrid-neural
- Concatenation des embedings et du tfidf
- chercher de nouvelles méthodes de régularisation pour les réseaux récurrents
- tester les CNN : https://www.kaggle.com/sanikamal/text-classification-with-python-and-keras
- ajouter une couche de positinal encoding : https://github.com/kaushalshetty/Positional-Encoding
- Librairie à essyaer rapidement :
    - text-classification-keras : https://pypi.org/project/text-classification-keras/
    - pytext :  https://github.com/facebookresearch/pytext

- approche avec des emmbedings déjà entrainés : 
    - https://adventuresinmachinelearning.com/word2vec-keras-tutorial/
    - https://medium.com/@ppasumarthi_69210/word-embeddings-in-keras-be6bb3092831
    